# **Вступление**

В качестве темы работы было выбрано ***исследование данных фитнес трекера***.

Данные были выгружены из приложения ***samsung health***.

Период использования и ***срок***, за который удалось получить данные: ***чуть больше года***.

# **Анализ структуры данных**

В результате просмотра и анализа данных, полученных из распакованного ***архива***, удалось выяснить следующее:


*   Есть данные в ***csv*** и отдельная папка с ***json***.
*   ***json*** хранят информацию о днях, разделённую на ***часы***. Это та же информация, что можно найти и в csv, только разбитая на более мелкие промежутки времени.
*   ***csv*** хранят в основном данные, сгруппированные по ***дням***, в некоторых местах по периодам активности, а также информацию профиля пользователя и различные таблицы, с которыми связаны основные (например, расшифровка типов активности).

# **Формирование датасета**

Для итогового датасета были использованы данные всего из ***2 входных файлов***.

Почему так мало:

*   Остальные файлы хранят информацию профиля или служебную информацию или дублировали данные из этих 2 файлов, разбивая её на более мелкие периоды.

В этих 2 файлах были ***пропущены***:

*   Столбцы, которые полностью ***пустые*** (floor_count всегда 0.0).
*   Столбцы, которые хранят ***одинаковую*** информацию во всех случаях (например, movement_type всегда 0).
*   Столбцы со ***служебной*** информацией (например, время добавления данных или тип записывающего устройства)

Таблицы были ***связаны*** по колонке ***day_time***. Поскольку в двух файлах эти данные хранились в разных форматах, они были приведены к общему unix datetime в миллисекундах.

Почему так можно:

*   Данные в этом столбце хранят всегда дату, за которую записаны данные и время 00:00:00.
*   Эти данные ***уникальны*** для каждой строки в обоих файлах (во втором файле есть исключение, но проверка показала, что это случайно записанные дублирующие данные, код это предусматривает).


# *Датасет*

Итоговый датасет включает в себя следующие данные (группировка по дням: столбец day_time):


*   exercise_time
*   step_count
*   exercise_calorie_target
*   active_time
*   target
*   dynamic_active_time_target
*   exercise_time_target
*   longest_active_time
*   score
*   move_hourly_count
*   move_hourly_target
*   distance
*   calorie
*   run_time
*   walk_time
*   longest_idle_time
*   day_time
*   total_exercise_calories
*   rest_calorie
*   active_calorie


# **Шаги для выполнения**

1.	Загрузить в среду выполнения следующие файлы:

*   com.samsung.shealth.activity.day_summary.20251007214549.csv
*   com.samsung.shealth.calories_burned.details.20251007214549.csv

2.	Запустить код
3.	В качестве input directory указать директорию относительно /content, в которой содержатся загруженные файлы
4.	В качестве output directory указать директорию относительно /content, в которой нужно сохранить результирующий файл
5.	Если код отработал верно и в конце есть сообщение “data preparing operations successfully finished!”, то результирующая информацию будет записана в файл с названием “fitness-tracker-data.csv”.

In [6]:
import os
import csv
import pandas

fullData = dict ()
inpDir = input ("enter input directory:")
outDir = input ("enter output directory:")
outputFileName = "fitness-tracker-data.csv"

try :
    print ("\nreading file with common info...")
    fileName = "com.samsung.shealth.activity.day_summary.20251007214549.csv"
    try :
        with open (os. path. join (inpDir, fileName), mode = "r") as inputFile :
            reader = csv. reader (inputFile)
            rowIndex = 0
            for row in reader :
                if rowIndex == 0 or rowIndex == 1 :
                    rowIndex += 1
                    continue
                if len (row) != 34 :
                    print ("row", rowIndex, "will be skipped due to wrong structure")
                    rowIndex += 1
                    continue
                if row [32] == 0 or row [32] == "" :
                    print ("row", rowIndex, "will be skipped due to absent time data")
                    rowIndex += 1
                    continue
                dayTime = str (int (pandas. to_datetime (row [32]). timestamp () * 1000))
                if dayTime in fullData :
                    print ("row", rowIndex, "will be skipped due to duplicating data")
                    rowIndex += 1
                    continue
                #              "exercise_time"           "step_count"                "exercise_calorie_target"
                rowData = [row [3],                  row [4],                    row [5],
                #         "active_time"             "target"                    "dynamic_active_time_target"
                           row [6],                  row [7],                    row [14],
                #         "exercise_time_target"    "longest_active_time"       "score"
                           row [15],                 row [17],                   row [18],
                #         "move_hourly_count"       "move_hourly_target"        "distance"
                           row [19],                 row [21],                   row [22],
                #         "calorie"                 "run_time"                  "walk_time"
                           row [24],                 row [27],                   row [29],
                #         "longest_idle_time"       "day_time" to unix ms
                           row [30],                 dayTime]
                fullData [dayTime] = rowData
                rowIndex += 1

    except FileNotFoundError:
        print ("seems like file", fileName, "was not found in input directory", inpDir)
        raise

    print ("\nreading file with calorie info...")
    fileName = "com.samsung.shealth.calories_burned.details.20251007214549.csv"
    try :
        with open (os. path. join (inpDir, fileName), mode = "r") as inputFile :
            reader = csv. reader (inputFile)
            rowIndex = 0
            for row in reader :
                if rowIndex == 0 or rowIndex == 1 :
                    rowIndex += 1
                    continue
                if len (row) != 18 :
                    print ("row", rowIndex, "will be skipped due to wrong structure")
                    rowIndex += 1
                    continue
                if row [16] == 0 or row [16] == "" :
                    print ("row", rowIndex, "will be skipped due to absent time data")
                    rowIndex += 1
                    continue
                dayTime = row [16]
                if not dayTime in fullData :
                    print ("row", rowIndex, "will be skipped due to absent common data")
                    rowIndex += 1
                    continue
                if len (fullData [dayTime]) == 20 :
                    print ("row", rowIndex, "will be skipped due to duplicating data")
                    rowIndex += 1
                    continue
                #        "total_exercise_calories"  "rest_calorie"              "active_calorie"
                rowData = [row [4],                  row [8],                    row [12]]
                fullData [dayTime]. extend (rowData)
                rowIndex += 1

    except FileNotFoundError :
        print ("seems like file", fileName, "was not found in input directory", inpDir)
        raise

    print ("\nwrite output file...")
    with open (os. path. join (outDir, outputFileName), mode = "w") as outputFile :
        writer = csv. writer (outputFile)
        headerRow = ["exercise_time",                "step_count",               "exercise_calorie_target",
                     "active_time",                  "target",                   "dynamic_active_time_target",
                     "exercise_time_target",         "longest_active_time",      "score",
                     "move_hourly_count",            "move_hourly_target",       "distance",
                     "calorie",                      "run_time",                 "walk_time",
                     "longest_idle_time",            "day_time",                 "total_exercise_calories",
                     "rest_calorie",                 "active_calorie"]
        writer. writerow (headerRow)
        writer. writerows (fullData. values ())

    print ("\ncreating dataset...")
    dataFrame = pandas. read_csv (os. path. join (outDir, outputFileName))

    print ("\ndata preparing operations successfully finished!")

except Exception as e :
    print ("\nsome exceptioin occured while processing data", e)

enter input directory:fitness
enter output directory:fitness
reading file with common info...

reading file with calorie info...
row 92 will be skipped due to duplicating data
write output file...
creating dataset...
data preparing operations successfully finished!
